In [ ]:
# Import libraries

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import category_encoders as ce
from math import sqrt

%matplotlib inline

pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 300)

from sklearn.preprocessing import OneHotEncoder, Imputer, StandardScaler, MinMaxScaler, normalize
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV,  cross_val_score
from sklearn.metrics import mean_squared_error

from sklearn_pandas import CategoricalImputer

import os
#print(os.listdir("../input"))

# Basic Workflow for Machine Learning
## By Jeff Hale

## Step 1: Write out your goal

Start with the big picture. Why are you interested in looking at some data and making a predictive model? Hoping to figure out how to beat the stock market? Want to predict where Malaria will show up? You've got something in mind. It's your motivating goal.

For this project we're documenting a machine learning workflow for a regression task. In particular, predicting housing prices with the [Ames Housing Dataset](https://ww2.amstat.org/publications/jse/v19n3/decock.pdf). We want to minimize the Root-Mean-Squared-Error (RMSE) between the logarithm of the predicted value and the logarithm of the observed sales price. 

We'll do some data cleaning and feature engineering and then explore a variety of machine learning algorithms. We'll try some OLS regression variations and then try KNN  and decision-tree-based algorithms.  

## Step 2: Explore variable data information
Write out the variables with definitions and thoughts in a google sheet. [Here's mine](https://docs.google.com/spreadsheets/d/106ZP2r97yRkkTbBqV9oEt00XNnjomhj3BvIaCNaeWlk/edit?usp=sharing) for this data set. This idea came from [ Pedro Marcelino's popular Kernel](https://www.kaggle.com/pmarcelino/comprehensive-data-exploration-with-python).

This exercise was very valuable. Insights:
1. There is probably a lot of duplication of information in the sense of very high correlation in the data. We'll probably be selecting a limited number of features to make a better model.
2. A lot of variables probably don't have much effect on the price of housing. We'll probably be selecting a limited number of features to make a more parsimonious regression model.
3. A *SaleType* of *COD* is likely to be important because a *Court Officer Deed/Estate* would be expected to substantially reduce the sale price of a property..
4.  *SaleCondition* categories that aren't *Normal* are likely to have large effects to the price.
4. A good deal of the data is ordinal.


## Ordinal Data

Interval and ratio data types are made of equally spaced quantities. For example, one cup of milk is exactly twice the amount of two cups of milk. Five cups of milk is exactly five times the amount of one cup of milk. (Ratio data has a meaningful zero point, but both data types can generally be used similarly in machine learning).

Ordinal data is in rank order, but not necessarily evenly spaced. For example, a movie rating of two stars is higher than a movie rating of one star, and a movie rating of five stars is higher than two stars. But we can't say that a movie that has five stars is five times better than a movie with a one star rating.

Nominal data doesn't have any clear numeric value scale associated with it. For example, if we are looking at the effect of shirt color on test scores, whether a shirt is blue or red doesn't have a numeric scale associated with being blue or red.

I hadn't previously come across methods for preserving the informational value in ordinal data in machine learning, so I went down the proverbial rabbit hole researching options.

There isn't a lot of info on the treadtment of ordinal data in machiine learning, which I find surprising because it's so common. Here's what I found

1. One hot encoding the categories is the predominent method for handling ordinal data. 
2. Decision tree based models might find the ordinal relationship naturally, or they might not depending upon their parameter settings. Seems better not to leave it to chance. A bagging algorithm with decision trees, such as a Random Forest Regressor or Randome Forest Classifier should be more likely to find the orindal relationship because it runs many models.
3. The categories could be treated as interval data, although they are strictly not because the space between each category is not likely to be exactly the same. This is a fairly easy way to go about coding the data and should result in better outcomes than treating the data as categorical.
4. The categories can be treated as binary data and this might work well in some cases.
5. There's a very promising approach outlined in a[ 2106 paper](https://www.aaai.org/ocs/index.php/AAAI/AAAI16/paper/view/12328) by researchers at USC. They have a method for estimating the size of the intervals between the categories. Turning this method into a sklearn package would be a cool project.
6. [Category_encoder package](http://contrib.scikit-learn.org/categorical-encoding/) makes this process easeir and was used by [one Kaggler ](https://www.kaggle.com/kilaorange/keras-deep-learning-approach)to transform the dataset into binary encoded data. 
7. Will McGinnis is the primary author of the category_encoder package and has a nice [blog](http://www.willmcginnis.com/2016/12/18/basen-encoding-grid-search-category_encoders/) where he explains some of his research evaluating different categorical encoding methods with different data sets. 


I'm going to treat the ordinal data as intervals for this project because that seems like a reasonable time/result trade off and leave further explorations for other projects.

I plan to make another kernel exploring methods for  transforming categorical data as part of preprocessing steps in a pipeline.

In [ ]:
# load the data
train_data = pd.read_csv("../input/train.csv")

## Step 3: Preliminarily data investigation
Run info, describe, and head methods to get a better feel for the data and some basic descriptive statistics.

In [ ]:
print(train_data.shape)
print(train_data.describe())

In [ ]:
print(train_data.info())

In [ ]:
print(train_data.head())

## Step 4: Let's make a very basic model with only square feet feature predicting the sale price.
We'll drop the null values and then fit a linear regression model.

In [ ]:
sqft = pd.DataFrame(train_data['GrLivArea'].values.reshape(-1,1), columns = ['GrLivArea'])

In [ ]:
sqft.describe()

In [ ]:
sqft.isnull().any()

In [ ]:
BX_train, BX_test, by_train, by_test = train_test_split(sqft, train_data['SalePrice'], test_size = .4, random_state = 34)
blr = LinearRegression()
blr.fit(BX_train, by_train)
predictions = blr.predict(BX_test)

rmse = sqrt(mean_squared_error(by_test, predictions))
print("score = {0:,.0f}".format(rmse))

## Step 5: Let's add some other numeric features and try a few more models
That RMSE isn't awful, but it's a bit high. Let's see if we can get it down by adding a few other predictors that are likely to have a large influence on the sale price.

In [ ]:
five_cols = ['GrLivArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd']
five_features = train_data.loc[:, five_cols]
print(five_features.head(3))

In [ ]:
# function to make models, fit, predict with a list of columns as a parameter

def make_model(cols):
    X_train, X_test, y_train, y_test = train_test_split(train_data[cols], train_data['SalePrice'], test_size = .4, random_state = 34)
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    predictions = lr.predict(X_test)

    score = sqrt(mean_squared_error(y_test, predictions))
    print("Columns: {0}, score: {1:,.0f}".format(cols, score))
make_model(five_cols)

RMSE dropped to 45k. Now we're cooking!

Let's try the same five features with a basic KNN model

In [ ]:
# better function to make models, fit, predict that can take a model type as a parameter

def make_model(cols, mod_type):
    X_train, X_test, y_train, y_test = train_test_split(train_data[cols], train_data['SalePrice'], test_size = .4, random_state = 34)
    mod = mod_type
    mod.fit(X_train, y_train)
    predictions = mod.predict(X_test)

    score = sqrt(mean_squared_error(y_test, predictions))
    print("Columns: {0}, score: {1:,.0f}".format(cols, score))
make_model(five_cols, KNeighborsRegressor())

Not so good with the default KNN parameters.  Let's make a loop to try a bunch of models with their preset parameters.

In [ ]:
mods = [Lasso(), Ridge(), GradientBoostingRegressor(), AdaBoostRegressor()]

for mod in mods:
    make_model(five_cols, mod)

Lasso is just limiting the number of features, which we already limited, so not seeing an improved score from that. Ridge is about the same score. The boosted regressors are about the same.  I think it's time to add some categorical data, clean some data, transform some data, and feature engineer.

## Step 6: Basic data visualization 
Make a correlation plot, heatmap, etc. to see what you have time to see.

In [ ]:
sns.pairplot(five_features) 
plt.show()

In [ ]:
y_df = train_data['SalePrice'].to_frame()
five_features['sale_price'] = y_df
#print(five_features)
correlation = five_features.corr()
print(correlation)
sns.heatmap(correlation)

## Step 7: Add ordinal features
Let's seperate the column types into numeric, ordinal, and categorical data types.

In [ ]:
# make lists of the numerical and string variable categories
numeric_cols = []
string_cols = []    # ordinal and categorical


for col in train_data.columns:
    # True integer or float columns
    if (train_data.dtypes[col] == np.int64 or train_data.dtypes[col] == np.int32 or train_data.dtypes[col] == np.float64):
        numeric_cols.append(col)
    # Categorical and oridnal columns
    if (train_data.dtypes[col] == np.object):  
        string_cols.append(col)
        
print(numeric_cols)
print(string_cols)
            

In [ ]:
y = train_data['SalePrice']
X = train_data.drop(columns = ["SalePrice"], axis = 1)

In [ ]:
print(X.columns)

In [ ]:
# List of ordinal variables with mappings
# For all of these, the first values given are the highest and best
ord_cols = [
"ExterQual",
"ExterCond",
"BsmtQual",
"BsmtCond",
"BsmtExposure",
"BsmtFinType1",
"BsmtFinType2",
"HeatingQC",
"Electrical",
"KitchenQual",
"Functional",
"FireplaceQu",
"GarageQual",
"GarageCond",
"PoolQC",
"Fence"]

In [ ]:
string_data = X[string_cols]
print(string_data.columns) #includes ordinal data
cat_cols = []

for col in string_data:
    if col not in ord_cols:
        cat_cols.append(col)
    
print(cat_cols)
print(type(cat_cols))

## Step 8: Impute missing values

Before we start imputing values, we need to make sure that our testing set data isn't contaminating our imputed values, so we'll break the data in to training and testing sets now.

In [ ]:
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(X, y, test_size = .3, random_state = 34)

In [ ]:
# impute missing values for string data (ordinal and categorical)
s_imputer = CategoricalImputer()

X_train_string = X_train_s[string_cols].values
X_train_string = s_imputer.fit_transform(X_train_string)
X_train_string = pd.DataFrame(X_train_string, columns = string_cols)

print(X_train_string.shape, X_train_string.head(3))

X_test_string = X_test_s[string_cols].values
X_test_string = s_imputer.transform(X_test_string)
X_test_string = pd.DataFrame(X_test_string, columns = string_cols)

# all categorical and ordinal missing data points have now been replaced with the most common value in each feature
# still need to impute the values for the actual interval data. We'll do that with the most frequent value, too.
# then we need to combine into one whole dataframe
# first time I did this there were some cases where the most common case is not-existent, 
# so need to either delete those variables or somehow deal with them

X_train_cat = X_train_string[cat_cols]
X_test_cat = X_test_string[cat_cols]


### We're going to treat the ordinal data as interval data for now. Later we might build some models with the ordinal data as categorical data.

In [ ]:
# imputing missing values with most freqent values for the true interval columns
n_imputer = Imputer(missing_values='NaN', copy = True, strategy = 'most_frequent')

numeric_cols.remove('SalePrice')

X_train_num = X_train_s[numeric_cols]
X_train_num = n_imputer.fit_transform(X_train_num)
X_train_num = pd.DataFrame(X_train_num, columns = numeric_cols)

X_test_num = X_test_s[numeric_cols]
X_test_num = n_imputer.transform(X_test_num)
X_test_num = pd.DataFrame(X_test_num, columns = numeric_cols)

print(X_test_num.head(3))


In [ ]:
# mapping string ordinal data 
ord_cols_map = [
    {"col":"ExterQual",    
    "mapping": [
        ('Ex',5), 
        ('Gd',4), 
        ('TA',3), 
        ('Fa',2), 
        ('Po',1), 
        ('NA',np.nan)
    ]},
    {"col":"ExterCond",
    "mapping": [
        ('Ex',5), 
        ('Gd',4), 
        ('TA',3), 
        ('Fa',2), 
        ('Po',1), 
        ('NA',np.nan)
    ]}, 
    {"col":"BsmtQual",
    "mapping": [
        ('Ex',5), 
        ('Gd',4), 
        ('TA',3), 
        ('Mn',2), 
        ('No',1), 
        ('NA',np.nan)   
    ]},
    {'col': "BsmtCond",
    'mapping': [
        ('Ex',5), 
        ('Gd',4), 
        ('TA',3), 
        ('Fa',2), 
        ('Po',1), 
        ('NA', np.NaN)
    ]},
    {'col': "BsmtExposure",
    'mapping': [
        ('Gd', 4),
        ('Av', 3),
        ('Mn', 2),
        ('No', 1),
        ('NA', np.NaN)
    ]},
    {'col': "BsmtFinType1",
    'mapping': [
        ('GLQ', 6),
        ('ALQ', 5),
        ('BLG', 4),
        ('Rec', 3),
        ('LwQ', 2),
        ('Unf', 1),
        ('NA', np.NaN)
    ]},
    {'col': "BsmtFinType2",
    'mapping': [
        ('GLQ', 6),
        ('ALQ', 5),
        ('BLG', 4),
        ('Rec', 3),
        ('LwQ', 2),
        ('Unf', 1),
        ('NA', np.NaN)
    ]},
     {"col":"HeatingQC",
     "mapping": [
        ('Ex',5), 
        ('Gd',4), 
        ('TA',3), 
        ('Fa',2), 
        ('Po',1), 
    ]}, 
    {"col": "Electrical",
    "mapping":[
        ('SBrkr', 4),
        ('FuseA', 3),
        ('FuseF', 2),       
        ('FuseP', 1),
    ]},
    {"col":"KitchenQual",
    "mapping": [
        ('Ex',5), 
        ('Gd',4), 
        ('TA',3), 
        ('Fa',2), 
        ('Po',1), 
    ]}, 
    {'col':"Functional",
     'mapping': [
        ('Typ', 8),
        ('Min1', 7),
        ('Min2', 6),   
        ('Mod', 5),
        ('Maj1', 4),
        ('Maj2', 3),
        ('Sev', 2),
        ('Sal', 1)
     ]},
    {"col":"FireplaceQu",
    "mapping": [
        ('Ex',5), 
        ('Gd',4), 
        ('TA',3), 
        ('Fa',2), 
        ('Po',1), 
        ('NA',np.NaN)
    ]}, 
    {'col': "GarageQual",
     "mapping": [
        ('Ex',5), 
        ('Gd',4), 
        ('TA',3), 
        ('Fa',2), 
        ('Po',1), 
        ('NA',np.NaN)
     ]},
    {'col': "GarageCond",
     "mapping": [
        ('Ex',5), 
        ('Gd',4), 
        ('TA',3), 
        ('Fa',2), 
        ('Po',1), 
        ('NA',np.NaN)
    ]},
    {'col': "PoolQC",
     "mapping": [
        ('Ex',5), 
        ('Gd',4), 
        ('TA',3), 
        ('Fa',2), 
        ('Po',1), 
        ('NA',np.NaN)
    ]},
    {'col': "Fence", 
     "mapping": [
        ('GdPrv', 5),
        ('MnPrv', 4),
        ('GdWo', 3), 
        ('MnWw', 2),
        ('NA', np.NaN)
    ]}]

# use the Ordinal Encoder to map the ordinal data to interval and then fit transform 
encoder = ce.OrdinalEncoder( return_df = True, mapping = ord_cols_map)  #NaNs get -1
X_train_ord = encoder.fit_transform(X_train_string)
X_test_ord = encoder.transform(X_test_string)

# ord has ordinal and categorical data in it!
print(X_train_ord.head(2))  

Let's fit a model with our five numeric features and our ordinal encoded features encoded as interval data  and unencoded categorical data. 

In [ ]:
# X_train_ord is ordinal cols training X df and categorical 
# X_train_cat is categorical cols training X df
# X_train_num  is numeric cols training X df

X_train_ord.reset_index(drop=True)
X_train_cat.reset_index(drop = True)


In [ ]:
print(X_train_ord.columns, X_train_num.columns)

X_train_all = pd.concat([X_train_ord, X_train_num], axis = 1)
print(X_train_all.shape)
print(X_train_all.columns)

In [ ]:
X_test_all = pd.concat([X_test_ord, X_test_num], axis = 1)

print(X_test_all.shape, X_test_all.head(2))

## Step 9: One hot encode the categorical data and make a model that includes it

In [ ]:
print(cat_cols)

In [ ]:
print(X_train_all.columns)

In [ ]:
# nominal categorical data gets one-hot-encoder treatment
# could do with sklearn one hot endcoder
print(X_train_all.head(2))


X_train = pd.get_dummies(data= X_train_all, prefix = "Dummy_", columns = cat_cols, drop_first= True)
print(X_train.shape)
X_train = X_train.drop(cat_cols, axis = 1)
print(X_train.head(3))

In [ ]:
# same treatment for test features
X_test = pd.get_dummies(data= X_test_all, prefix = "Dummy_", columns = cat_cols, drop_first= True)
X_test = X_test.drop(cat_cols, axis = 1)

In [ ]:
print(X_train.columns)

In [ ]:


# because the test data doesn't have as many categories present it doesn't one hot encode as many dummy columns

# Let's align the data frames - we only use the dummies in the training data to build our mode - 
# -we can't learn from the dummies only in the test data

X_train, X_test = X_train.align(X_test, join = "left", fill_value = 0, axis = 1) 

print(X_train.shape)
print(X_test.shape)
print(X_train.columns)
X_train.head(3)
# there are still -1s ins ordinal data !!!!, thought those should have been filled above and 4 fences

That's' a lot of columns, we'll look at trimming that down to the most important ones in a bit. Let's fit a linear regression model and now and see what we get.

In [ ]:
# fit and run a linear regression model with all columns

mod = LinearRegression()
mod.fit(X_train, y_train_s)
predictions = mod.predict(X_test)

print(predictions[:5])

rmse = sqrt(mean_squared_error(y_test_s, predictions))
print("score: {0:,.0f}".format(rmse))

This looks wierd. Maybe too many features. The RMSE has gone up! All the work and we aren't doing better. Let's try just picking a few variables with Lasso.

In [ ]:
mod = Lasso()
mod.fit(X_train, y_train_s)
predictions = mod.predict(X_test)

print(predictions[:5])

rmse = sqrt(mean_squared_error(y_test_s, predictions))
print("score: {0:,.0f}".format(rmse))

In [ ]:
mod = KNeighborsRegressor()
mod.fit(X_train, y_train_s)
predictions = mod.predict(X_test)

print(predictions[:5])

rmse = sqrt(mean_squared_error(y_test_s, predictions))
print("score: {0:,.0f}".format(rmse))

That's getting worse. Ok, let's try a tree-based model.

In [ ]:
mod = GradientBoostingRegressor()
mod.fit(X_train, y_train_s)
predictions = mod.predict(X_test)

print(predictions[:5])

rmse = sqrt(mean_squared_error(y_test_s, predictions))
print("score: {0:,.0f}".format(rmse))

That's the best yet, by a small amount. Hmm. Scaling the data should help regression models, but not likely to have much of an effect on tree-based models.

## Step 10: Transform and scale
Important for some machine learning algorithms, including regression-based models, Kernel SVMs, Neural Nets. 

> Many machine learning algorithms in sklearn requires standardized data which means having zero mean and unit variance.
> Standardization (or Z-score normalization) is the process where the features are rescaled so that they’ll have the properties of a standard normal distribution with μ=0 and σ=1, where μ is the mean (average) and σ is the standard deviation from the mean. 

Source: https://www.analyticsvidhya.com/blog/2016/07/practical-guide-data-preprocessing-python-scikit-learn/

We'll log transform and scale the true interval variables to improve the model output.

Log transform to improve lineariety first so it gets to use the full variance as part of the operation.
Then normalize the data so all features are on a similar scale.
Then add polynomials and interactions.

We'll have to break apart the data_types and then retransform them. Dummy encoded categorical data won't be affected here - we'll focus on the numerical and ordinal data.


In [ ]:
# drop duplicate columns
X_train = X_train.loc[:,~X_train.columns.duplicated()]
X_test = X_test.loc[:,~X_test.columns.duplicated()]

In [ ]:
print(X_train)

In [ ]:
# making all true interval data positive for log transformation
X_train[numeric_cols] += 2
X_train[ord_cols] += 2

X_test[numeric_cols] += 2
X_test[ord_cols] += 2

In [ ]:
# log transform the true interval variables to improve the model - not sure if we'll transform ordinal data - I think not
print(X_train.columns)
#X_train[numeric_cols] = X_train[numeric_cols].apply(np.log)

In [ ]:
print(X_train)

In [ ]:
# Scale data
scaler = StandardScaler()
scaler.fit_transform(X_train)

scaler.transform(X_test)

print(X_train.head())

In [ ]:
mod = GradientBoostingRegressor()
mod.fit(X_train, y_train_s)
predictions = mod.predict(X_test)

print(predictions[:5])

rmse = sqrt(mean_squared_error(y_test_s, predictions))
print("score: {0:,.0f}".format(rmse))


# add polynomials/interactions


A slight improvement in our best score. Hopefully we aren't overfitting. Let's submit to kaggle and see how we do.

In [ ]:
## submit to kagl

## Step 11: Bad outlier and abnormal data removal

> Note that outliers are not necessarily "bad" data-points; indeed they may well be the most important, most information rich, part of the dataset. Under no circumstances should they be automatically removed from the dataset. Outliers may deserve special consideration: they may be the key to the phenomenon under study or the result of human blunders.

[http://www.physics.csbsju.edu/stats/box2.html](http://www.physics.csbsju.edu/stats/box2.html)

However, after consideration you may still want to remove the outliers. Our aims with a model are pragmatic - we want the model that will best generalize the best to new data. I suggest testing your model with and without the outliers excluded.

Obviously if the outliers are important to the phenomenon you are studying, don't get rid of them.

In this case, we'll exclude the following outliers.


Probably get rid of abnormal sales, too.


## Step 11: Make a more sophisticated model building process with more advanced algorithms
Let's try a few different models in a gridsearchCV  and, if too slow, randomsearchCV. 
Either make a pipeline or a number of functions to make switching out models easier.
We could try out LightGBH, which maybe the performance model likely to work best for regression tasks.


## Step 12: Test against holdout data 
Not a bad idea to do this earlier, to make sure we aren't overfitting. We can submit a few models to the Kaggle competion and see how the RMSE changes.

## Step 13: Iterate
Engineer more features -> Fit Models -> Score -> Submit

`## To do:

### Time series
We've got a few dates in the data. More recent sales are likely to be more valuable to the model. We'll look at this later. 

Our google sheet shows that all the data-related variables are coded as integers.

### Use KNN to better impute values
use this method:
[https://towardsdatascience.com/the-use-of-knn-for-missing-values-cf33d935c637](https://towardsdatascience.com/the-use-of-knn-for-missing-values-cf33d935c637)

### Binary encode the categorical variables
See how that performs compared to treading the categorical data as interval data

# Lessons learned
I should separate the interval, ordinal, and categorical data right away.
